In [ ]:
import numpy as np
from numpy import random
import pandas as pd
from pandas.core.series import Series
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import random
import statistics
import time
import gc
import scipy.stats as st
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader

In [ ]:
etfs = ['SPY', 'AGG', 'VNQ', 'VEU', 'ITOT']
def get_price_data(train_start='2010-01-01',train_end='2014-12-31'):
    filepath='./processed_data.csv'
    df = pd.read_csv(filepath)
    # etfs = list(set(df.tic))
    # train_start = '2010-01-01'
    # train_end = '2014-12-31'
    price_data = []
    for tic in etfs:
        tmp_df = df[(df['tic']==tic)&(df['date']>=train_start)&(df['date']<=train_end)]
        tmp_df = tmp_df.reset_index(drop=True)
        price_data.append(tmp_df)
    return price_data

In [ ]:
def prepare_train_data(windowx=63,windowy=21):
    price_data = get_price_data()
    all_x_tr = []
    all_x_te = []
    all_y_tr = []
    all_y_te = []
    for df in price_data:
        x = []
        y = []
        now_tic = df['tic'][0]
        for i in range(len(df)):
            if i+windowx+windowy>len(df):
                break
            tmp_x = np.array(df[df.columns[3:]][i:i+windowx]).tolist()
            tmp_y = (df['close'][i+windowx+windowy-1] - df['close'][i+windowx-1])/df['close'][i+windowx-1]
            x.append(tmp_x)
            y.append(tmp_y)
        # all_x[now_tic] = x
        # all_y[now_tic] = y
        all_x_tr.append(x[:880])
        all_x_te.append(x[880:])
        if now_tic=='SPY':
            continue
        all_y_tr.append(y[:880])
        all_y_te.append(y[880:])
    return all_x_tr,all_y_tr,all_x_te,all_y_te

def get_predict_data(pred_start,pred_end,windowx=63):
    price_data = get_price_data(train_start=pred_start,train_end=pred_end,windowx=windowx)
    for df in price_data:
        x = []
        y = []
        # now_tic = df['tic'][0]
        tmp_x = np.array(df[df.columns[3:]][0:len(df)]).tolist()
        x.append(tmp_x)
    return x

In [ ]:
class Attention_lstm(nn.Module):
    def __init__(self,units,input_shape):
        super(Attention_lstm, self).__init__()
        self.input_size
        self.hidden_dim
        self.num_layers
        self.lstm_en = nn.LSTM(self.input_size, self.hidden_dim, num_layers = self.num_layers)
        self.lstm_de = layers.SimpleRNN(units,dropout=0.2,unroll=True)
        self.attention = layers.Attention()
        self.fc = layers.Dense(units)
        self.out = layers.Dense(1)
        
    def call(self, inputs, training=None):
        x = self.lstm_en(inputs)
        x = self.attention([x, x])
        x = self.lstm_de(x)
        x = self.fc(x)
        # out = self.out(x)
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self,num_heads,input_dim,units):
        super(Transformer, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=input_dim)
        self.fc1 = layers.Dense(4*units,activation='relu')
        self.fc2 = layers.Dense(1*units,activation='relu')
        self.attention = layers.Attention()
    def call(self, inputs, training=None):
        h_tilde = self.attention([inputs, inputs])
        h2 = h_tilde+inputs
        h3 = self.fc1(h2)
        h3 = self.fc2(h3)
        # print('h2',h2.shape,'h3',h3.shape)#h2 (4, 1, 30, 32) h3 (4, 1, 30, 32)
        out = tf.math.tanh(h2+h3)
        # print(out.shape)
        return out

In [ ]:
class MyModel(nn.Module):
    def __init__(self,units,input_shape,num_heads,input_dim):
        super(MyModel, self).__init__()
        self.units = units
        self.attlstm = Attention_lstm(units,input_shape)
        self.transf = Transformer(num_heads,input_dim,units)
        self.out = layers.Dense(1,activation='sigmoid')
        self.layernorm = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training=None):
        
        # temperal attention
        hidden_vec = []
        # print(inputs.shape)
        new_inputs = tf.transpose(inputs, perm=[1,0,2,3])
        for i in range(new_inputs.shape[0]):
            h = self.attlstm(new_inputs[i])
            # h = self.layernorm(h)
            hidden_vec.append(h)
        # print('hidden_vec',len(hidden_vec)) #hidden_vec 5
        # print('hidden_vec[0]',(hidden_vec[0]).shape) #hidden_vec[0] (30, 32)

        # context aggregate
        new_hidden_vec = []
        beta = 0.5
        for i in range(1,len(hidden_vec)):
            new_hidden_vec.append(hidden_vec[i]+beta*hidden_vec[0])
        # print('new_hidden_vec',len(new_hidden_vec)) # new_hidden_vec 4
        print('new_hidden_vec[0]',(new_hidden_vec[0]).shape) #new_hidden_vec[0] (30, 64)

        # data axis attention
        big_h = tf.concat(values=[new_hidden_vec],axis=0)
        big_h = tf.reshape(big_h,(-1,len(new_hidden_vec),self.units))
        print('big h',big_h.shape)#big h (30,4,64)
        hp = self.transf(big_h)
        print('hp',hp.shape)#hp (30,4,64)
        out = self.out(hp)
        print('out',out.shape)#out (30,4,1)
        return out